# Đề kiểm tra giữa kỳ

Họ và tên: Võ Nguyễn Phương Quỳnh - MSSV: 22127360

Lớp: 22KHTM1

## Mô tả dữ liệu

Tập Dữ liệu được sử dụng trong bài tập là **Student Performance Factors** được lấy từ [Kaggle](https://www.kaggle.com/datasets/lainguyn123/student-performance-factors). Tập dữ liệu cho biết một số nhân tố ảnh hưởng đến điểm số của học sinh.
Tập dữ liệu sau khi được xử lý gồm *6378 dòng* và *20 cột* với ý nghĩa từng cột như sau:


| Thuộc tính                 | Mô tả                                                                         |
|----------------------------|-------------------------------------------------------------------------------|
| Hours_Studied              | Số giờ học mỗi tuần.                                                          |
| Attendance                 | Tỷ lệ phần trăm số buổi học đã tham gia.                                      |
| Parental_Involvement       | Mức độ tham gia của phụ huynh vào việc học của học sinh (Low, Medium, High).  |
| Access_to_Resources        | Mức độ sẵn có của các tài nguyên giáo dục (Low, Medium, High).                |
| Extracurricular_Activities | Tham gia vào các hoạt động ngoại khóa (Yes, No).                              |
| Sleep_Hours                | Số giờ ngủ trung bình mỗi đêm.                                                |
| Previous_Scores            | Điểm số từ các kỳ thi trước.                                                  |
| Motivation_Level           | Mức độ động lực học tập của học sinh (Low, Medium, High).                     |
| Internet_Access            | Mức độ truy cập Internet (Yes, No).                                           |
| Tutoring_Sessions          | Số buổi học phụ đạo tham gia mỗi tháng.                                       |
| Family_Income              | Mức thu nhập của gia đình (Low, Medium, High).                                |
| Teacher_Quality            | Chất lượng giáo viên (Low, Medium, High).                                     |
| School_Type                | Loại hình trường học (Public, Private).                                       |
| Peer_Influence             | Ảnh hưởng của bạn bè đến kết quả học tập (Positive, Neutral, Negative).       |
| Physical_Activity          | Số giờ hoạt động thể chất trung bình mỗi tuần.                                |
| Learning_Disabilities      | Có sự hiện diện của khuyết tật học tập hay không (Yes, No).                   |
| Parental_Education_Level   | Trình độ học vấn cao nhất của cha mẹ (High School, College, Postgraduate).    |
| Distance_from_Home         | Khoảng cách từ nhà đến trường (Near, Moderate, Far).                          |
| Gender                     | Giới tính của học sinh (Male, Female).                                        |
| Exam_Score                 | Điểm kỳ thi cuối cùng.                                                        |

Tập dữ liệu sẽ được chia thành hai tệp tin là: `train.csv` và `test.csv` với số lượng bản ghi tương ứng là  5102 và 1276.

## Yêu cầu đề bài

Trong nội dung kiểm tra này các bạn sẽ xây dựng hai mô hình Decision Tree và mô hình Linear Regression để dự đoán cột điểm số `Exam_score` dựa trên các thuộc tính còn lại.

**Yêu cầu chung**
* Các hành vi gian lận sẽ bị đánh giá **0 điểm** trong phần giữa kỳ.

* Chỉ được phép sử dụng các thư viện `pandas` và `numpy`. Các bạn có thể sử dụng thêm các thư viện trực quan như `seaborn` hoặc `matplotlib` để trực quan dữ liệu.

* Dữ liệu huấn luyện mô hình sẽ lấy từ file `train.csv`

* Dữ liệu kiểm tra mô hình các bạn sẽ lấy từ file `test.csv`

* **Đối với mô hình Desion Tree**, dữ liệu chỉ gồm các cột `Parental_Involvement`, `Access_to_Resources`, `Extracurricular_Activities`, `Motivation_Level`, `Internet_Access`, `Family_Income`, `Teacher_Quality`, `School_Type`, `Peer_Influence`, `Learning_Disabilities`, `Parental_Education_Level`, `Distance_from_Home`, `Gender` và cột điểm số `Exam_score`

* **Đối với mô hình Linear Regression**, dữ liệu chỉ gồm `Hours_Studied`, `Attendance`, `Sleep_Hours`, `Previous_Scores`, `Tutoring_Sessions`, `Physical_Activity` và cột điểm số `Exam_score`các cột .

In [289]:
# YOUR CODE HERE
# import các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot') 
import matplotlib.patches as patches

### Câu 1 - Decision Tree

a. Viết chương trình xây dựng cây quyết định (theo mẫu) trên các tất cả thuộc tính (trừ `Exam_Score`). Xác định độ lỗi MSE (mean square error) của mô hình trên tập test.

b. Xây dựng 5 mô hình cây quyết định bằng việc chọn lần lượt 1 thuộc tính, 2 thuộc tính, 3 thuộc tính, 4 thuộc tính, 5 thuộc tính. Giải thích lý do lựa chọn các cột đó và tính toán độ lỗi MSE trên tập test.

In [290]:
# Xây dựng cây quyết định
class DecisionTree:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth
        self.tree = None
        
    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
            self.feature = feature       # Indes của feature để chia cây
            self.threshold = threshold   # Threshold
            self.left = left             # Con trái
            self.right = right           # Con phải
            self.value = value           # Value if leaf node (class prediction)
    
    def fit(self, X, y):
        """Fit the decision tree to the data."""
        self.tree = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(set(y)) == 1:
            return self.Node(value=self._most_common_label(y))

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:  # If no split is found, return a leaf
            return self.Node(value=self._most_common_label(y))
        left_idx = X[:, best_feature] < best_threshold
        right_idx = ~left_idx
        
        # Kiểm tra xem các chỉ số có ít nhất một mẫu không
        if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
            return self.Node(value=self._most_common_label(y))

        # Tạo các nút con bên trái và bên phải
        left = self._grow_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._grow_tree(X[right_idx], y[right_idx], depth + 1)
        return self.Node(best_feature, best_threshold, left, right)

    def _most_common_label(self, y):
        """Return the most common class label in the array y."""
        unique, counts = np.unique(y, return_counts=True)
        return unique[np.argmax(counts)]
    
    def _best_split(self, X, y):
        best_gini = float('inf')
        best_feature, best_threshold = None, None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gini = self._gini_impurity(X, y, feature, threshold)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def _gini_impurity(self, X, y, feature, threshold):
        left_idx = X[:, feature] < threshold
        right_idx = ~left_idx
        left_gini = self._gini(y[left_idx])
        right_gini = self._gini(y[right_idx])
        p_left = len(y[left_idx]) / len(y)
        p_right = len(y[right_idx]) / len(y)
        return p_left * left_gini + p_right * right_gini

    def _gini(self, y):
        counts = np.bincount(y)
        p = counts / len(y)
        return 1 - np.sum(p ** 2)

    def predict(self, X):
        """Predict the class for each instance in X."""
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.tree
        while node.value is None:  
            if inputs[node.feature] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

    def loss(self, y_pred, y):
        """Calculate zero-one loss between predicted labels y_pred and actual labels y."""
        return np.mean((y_pred - y) ** 2)

    def drawTree(self, node=None, depth=0):
        """Recursively prints a text-based tree structure."""
        """Recursively prints a text-based tree structure."""
        if node is None:
            node = self.tree

        if node.value is not None:
            print(f"{'|   ' * depth}Leaf: {node.value}")
        else:
            print(f"{'|   ' * depth}Feature {node.feature} < {node.threshold}")
            self.drawTree(node.left, depth + 1)
            self.drawTree(node.right, depth + 1)
        

In [291]:
# Các cột dữ liệu cần thiết cho bài toán cây quyết định
columns_decision_tree = [
    "Parental_Involvement", "Access_to_Resources", "Extracurricular_Activities", 
    "Motivation_Level", "Internet_Access", "Family_Income", "Teacher_Quality", 
    "School_Type", "Peer_Influence", "Learning_Disabilities", 
    "Parental_Education_Level", "Distance_from_Home", "Gender", "Exam_Score"
]

# Đọc dữ liệu từ file csv
train_data_dt = pd.read_csv("train.csv", usecols=columns_decision_tree)
test_data_dt = pd.read_csv("test.csv", usecols=columns_decision_tree)
train_data_dt

,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Motivation_Level,Internet_Access,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,Low,High,No,Medium,Yes,Medium,High,Private,Positive,Yes,High School,Moderate,Male,70
1,High,Medium,Yes,Low,Yes,Low,Medium,Public,Neutral,No,College,Moderate,Male,67
2,Medium,High,Yes,Medium,Yes,Medium,High,Public,Negative,No,High School,Moderate,Male,62
3,Medium,High,No,Medium,Yes,Low,Medium,Public,Neutral,No,Postgraduate,Near,Female,63
4,Medium,High,Yes,Low,No,Low,Medium,Public,Positive,No,High School,Near,Male,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5097,High,Low,No,High,Yes,Medium,Medium,Public,Positive,No,Postgraduate,Near,Male,70
5098,Low,Low,Yes,High,Yes,Low,Medium,Public,Positive,No,High School,Near,Male,71
5099,Low,High,Yes,Medium,Yes,High,Medium,Public,Neutral,No,Postgraduate,Moderate,Female,64
5100,Medium,High,Yes,Low,Yes,Medium,High,Public,Positive,No,College,Near,Female,75


In [292]:
# Chuyển dữ liệu về dạng numpy array
X_train = train_data_dt.drop(columns=["Exam_Score"]).values
y_train = train_data_dt["Exam_Score"].values
X_test = test_data_dt.drop(columns=["Exam_Score"]).values
y_test = test_data_dt["Exam_Score"].values

In [293]:
# Huấn luyện và đánh giá mô hình
tree_model = DecisionTree()
tree_model.fit(X_train, y_train)

# Dự đoán và tính MSE
y_pred = tree_model.predict(X_test)
mse = tree_model.loss(y_pred, y_test)

# In kết quả và vẽ cây
print("MSE on test set:", mse)
tree_model.drawTree()

MSE on test set: 19.739811912225704
Feature 1 < Low
|   Feature 10 < Postgraduate
|   |   Feature 11 < Near
|   |   |   Feature 5 < Medium
|   |   |   |   Feature 8 < Positive
|   |   |   |   |   Feature 6 < Medium
|   |   |   |   |   |   Feature 5 < Low
|   |   |   |   |   |   |   Feature 9 < Yes
|   |   |   |   |   |   |   |   Feature 3 < Low
|   |   |   |   |   |   |   |   |   Feature 2 < Yes
|   |   |   |   |   |   |   |   |   |   Leaf: 63
|   |   |   |   |   |   |   |   |   |   Leaf: 72
|   |   |   |   |   |   |   |   |   Feature 3 < Medium
|   |   |   |   |   |   |   |   |   |   Leaf: 69
|   |   |   |   |   |   |   |   |   |   Leaf: 63
|   |   |   |   |   |   |   |   Feature 0 < Low
|   |   |   |   |   |   |   |   |   Feature 3 < Medium
|   |   |   |   |   |   |   |   |   |   Leaf: 66
|   |   |   |   |   |   |   |   |   |   Leaf: 72
|   |   |   |   |   |   |   |   |   Leaf: 66
|   |   |   |   |   |   |   Feature 9 < Yes
|   |   |   |   |   |   |   |   Feature 12 < Male
|   |   | 

In [294]:
def calculate_mse(y_pred, y):
    """Tính độ lỗi MSE giữa nhãn dự đoán và nhãn thực tế."""
    # Đảm bảo rằng y_pred và y có cùng độ dài
    if len(y_pred) != len(y):
        raise ValueError("Chiều dài của y_pred và y phải giống nhau.")
    
    # Tính MSE
    mse = np.mean((y_pred - y) ** 2)
    return mse

In [295]:
mse_results = {}

for i in range(X_train.shape[1]):
    selected_feature = X_train[:, i].reshape(-1, 1)
    test_feature = X_test[:, i].reshape(-1, 1)

    tree = DecisionTree()
    tree.fit(selected_feature, y_train)

    y_pred = tree.predict(test_feature)

    mse = calculate_mse(y_pred, y_test)
    mse_results[columns_decision_tree[i]] = mse

sorted_mse = sorted(mse_results.items(), key=lambda x: x[1])

# Lấy 5 thuộc tính tốt nhất dựa trên độ lỗi MSE
features = sorted_mse[:5]
top_features = []

print("5 thuộc tính tốt nhất và độ lỗi MSE tương ứng:")
for feature, mse in features:
    print(f"{feature}: Độ lỗi MSE = {mse:.4f}")
    top_features.append(feature)
    


5 thuộc tính tốt nhất và độ lỗi MSE tương ứng:
Access_to_Resources: Độ lỗi MSE = 15.0400
Parental_Involvement: Độ lỗi MSE = 15.1520
Distance_from_Home: Độ lỗi MSE = 15.5870
Peer_Influence: Độ lỗi MSE = 15.7806
Internet_Access: Độ lỗi MSE = 15.9734


Ta xây dựng hàm tìm lỗi MSE cho từng thuộc tính, từ đó chọn ra 5 thuộc tính có lỗi MSE thấp nhất:
- Access_to_Resources: Độ lỗi MSE = 15.0400
- Parental_Involvement: Độ lỗi MSE = 15.1520
- Distance_from_Home: Độ lỗi MSE = 15.5870
- Peer_Influence: Độ lỗi MSE = 15.7806
- Internet_Access: Độ lỗi MSE = 15.9734
Chọn lần lượt 1, 2, 3, 4, 5 thuộc tính đầu tiên trong 5 thuộc tính này để xây dựng 5 cây Decision tree tương ứng

In [296]:
num_features = [1, 2, 3, 4, 5]

def tree_n_features(top_features, n, train_data_dt, test_data_dt, y_train, y_test):
    selected_features = top_features[:n]

    X_train_selected = train_data_dt[selected_features].values
    X_test_selected = test_data_dt[selected_features].values

    tree = DecisionTree()
    tree.fit(X_train_selected, y_train)

    y_pred = tree.predict(X_test_selected)
    mse = calculate_mse(y_pred, y_test)
    mse_results[n] = mse
    
    print(f"Mô hình với {n} thuộc tính: {', '.join(selected_features)} - Độ lỗi MSE = {mse:.4f}")
    tree.drawTree()

In [297]:
tree_n_features(top_features, 1, train_data_dt, test_data_dt, y_train, y_test)

Mô hình với 1 thuộc tính: Access_to_Resources - Độ lỗi MSE = 15.0400
Feature 0 < Low
|   Leaf: 68
|   Feature 0 < Medium
|   |   Leaf: 66
|   |   Leaf: 67


In [298]:
tree_n_features(top_features, 2, train_data_dt, test_data_dt, y_train, y_test)

Mô hình với 2 thuộc tính: Access_to_Resources, Parental_Involvement - Độ lỗi MSE = 15.0823
Feature 0 < Low
|   Feature 1 < Low
|   |   Leaf: 68
|   |   Feature 1 < Medium
|   |   |   Leaf: 68
|   |   |   Leaf: 67
|   Feature 0 < Medium
|   |   Feature 1 < Low
|   |   |   Leaf: 66
|   |   |   Feature 1 < Medium
|   |   |   |   Leaf: 66
|   |   |   |   Leaf: 66
|   |   Feature 1 < Low
|   |   |   Leaf: 68
|   |   |   Feature 1 < Medium
|   |   |   |   Leaf: 65
|   |   |   |   Leaf: 67


In [299]:
tree_n_features(top_features, 3, train_data_dt, test_data_dt, y_train, y_test)

Mô hình với 3 thuộc tính: Access_to_Resources, Parental_Involvement, Distance_from_Home - Độ lỗi MSE = 15.7625
Feature 0 < Low
|   Feature 2 < Near
|   |   Feature 1 < Low
|   |   |   Feature 2 < Moderate
|   |   |   |   Leaf: 67
|   |   |   |   Leaf: 67
|   |   |   Feature 2 < Moderate
|   |   |   |   Feature 1 < Medium
|   |   |   |   |   Leaf: 62
|   |   |   |   |   Leaf: 67
|   |   |   |   Feature 1 < Medium
|   |   |   |   |   Leaf: 68
|   |   |   |   |   Leaf: 67
|   |   Feature 1 < Low
|   |   |   Leaf: 68
|   |   |   Feature 1 < Medium
|   |   |   |   Leaf: 66
|   |   |   |   Leaf: 66
|   Feature 0 < Medium
|   |   Feature 1 < Low
|   |   |   Feature 2 < Moderate
|   |   |   |   Leaf: 64
|   |   |   |   Feature 2 < Near
|   |   |   |   |   Leaf: 66
|   |   |   |   |   Leaf: 66
|   |   |   Feature 1 < Medium
|   |   |   |   Feature 2 < Near
|   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   Leaf: 65
|   |   |   |   |   |   Leaf: 64
|   |   |   |   |   Leaf: 66
|   

In [300]:
tree_n_features(top_features, 4, train_data_dt, test_data_dt, y_train, y_test)

Mô hình với 4 thuộc tính: Access_to_Resources, Parental_Involvement, Distance_from_Home, Peer_Influence - Độ lỗi MSE = 16.7092
Feature 0 < Low
|   Feature 2 < Near
|   |   Feature 3 < Neutral
|   |   |   Feature 1 < Low
|   |   |   |   Feature 2 < Moderate
|   |   |   |   |   Leaf: 61
|   |   |   |   |   Leaf: 65
|   |   |   |   Feature 1 < Medium
|   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   Leaf: 63
|   |   |   |   |   |   Leaf: 64
|   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   Leaf: 67
|   |   |   |   |   |   Leaf: 67
|   |   |   Feature 1 < Low
|   |   |   |   Feature 3 < Positive
|   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   Leaf: 65
|   |   |   |   |   |   Leaf: 67
|   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   Leaf: 67
|   |   |   |   |   |   Leaf: 71
|   |   |   |   Feature 2 < Moderate
|   |   |   |   |   Feature 3 < Positive
|   |   |   |   |   |   Feature 1 < Medium
|   |   |   |   |   |   |   Leaf: 62

In [301]:
tree_n_features(top_features, 5, train_data_dt, test_data_dt, y_train, y_test)

Mô hình với 5 thuộc tính: Access_to_Resources, Parental_Involvement, Distance_from_Home, Peer_Influence, Internet_Access - Độ lỗi MSE = 17.5878
Feature 0 < Low
|   Feature 2 < Near
|   |   Feature 3 < Neutral
|   |   |   Feature 1 < Low
|   |   |   |   Feature 2 < Moderate
|   |   |   |   |   Feature 4 < Yes
|   |   |   |   |   |   Leaf: 65
|   |   |   |   |   |   Leaf: 61
|   |   |   |   |   Feature 4 < Yes
|   |   |   |   |   |   Leaf: 67
|   |   |   |   |   |   Leaf: 65
|   |   |   |   Feature 1 < Medium
|   |   |   |   |   Feature 4 < Yes
|   |   |   |   |   |   Leaf: 60
|   |   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   |   Leaf: 63
|   |   |   |   |   |   |   Leaf: 63
|   |   |   |   |   Feature 4 < Yes
|   |   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   |   Leaf: 68
|   |   |   |   |   |   |   Leaf: 69
|   |   |   |   |   |   Feature 2 < Moderate
|   |   |   |   |   |   |   Leaf: 67
|   |   |   |   |   |   |   Leaf: 67
|   |   |   Feature 4 < Y

### Câu 2 - Linear Regression

a. Xác định mô hình hồi quy tuyến tính $y = w_0 + w_1 * \text{Attendance} + w_2 * \text{Hours_Studied} + w_3 * \text{Previous_Scores}\^2 $ và độ lỗi mean square error (MSE) của mô hình.

b. Xây dựng 5 mô hình hồi quy tuyến tính sử dụng ít nhất 2 thuộc tính và giải thích lý do chọn mô hình hồi quy tuyến tính. Tính độ lỗi MSE của từng mô hình.

In [302]:
# Các cột dữ liệu cần thiết cho bài toán cây quyết định 
columns_linear_regression = ["Hours_Studied", "Attendance", "Sleep_Hours", "Previous_Scores", "Tutoring_Sessions", "Physical_Activity", "Exam_Score"]

# Đọc dữ liệu từ file csv
train_data_lr = pd.read_csv("train.csv", usecols=columns_linear_regression)
test_data_lr = pd.read_csv("test.csv", usecols=columns_linear_regression)
train_data_lr

,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score
0,18,95,7,96,2,4,70
1,16,89,8,58,2,3,67
2,16,69,6,55,1,1,62
3,11,65,7,78,2,3,63
4,21,95,8,57,1,3,69
...,...,...,...,...,...,...,...
5097,30,70,6,84,2,3,70
5098,24,90,9,97,3,4,71
5099,24,65,7,52,1,1,64
5100,30,91,7,95,1,3,75


In [303]:
train_data_lr['Previous_Scores_Squared'] = train_data_lr['Previous_Scores'] ** 2
test_data_lr['Previous_Scores_Squared'] = test_data_lr['Previous_Scores'] ** 2

X_train = train_data_lr[["Attendance", "Hours_Studied", "Previous_Scores_Squared"]].values
y_train = train_data_lr["Exam_Score"].values
X_test = test_data_lr[["Attendance", "Hours_Studied", "Previous_Scores_Squared"]].values
y_test = test_data_lr["Exam_Score"].values

train_data_lr.drop(columns=['Previous_Scores_Squared'], inplace=True)
test_data_lr.drop(columns=['Previous_Scores_Squared'], inplace=True)

In [304]:
def add_intercept(X):
    """Thêm cột intercept (cột 1) vào dữ liệu X."""
    return np.hstack((np.ones((X.shape[0], 1)), X))

def train_linreg(X, y):
    w = np.linalg.pinv(X.T @ X) @ X.T @ y
    return w

def predict_linreg(X, w):
    return X @ w

def loss_function(y_pred, y):
    return np.mean((y_pred - y) ** 2)


In [305]:
X_train = add_intercept(X_train)
X_test = add_intercept(X_test)

w = train_linreg(X_train, y_train)

y_pred = predict_linreg(X_test, w) 

mse_test = loss_function(y_pred, y_test)
mse_train = loss_function(predict_linreg(X_train, w), y_train)  # Just to compare with test set

print("Weights (w):", w.round(3))
print("Mean Squared Error (MSE) trên tập test:", mse_test)
print("Mean Squared Error (MSE) trên tập train:", mse_train)

Weights (w): [43.777  0.198  0.291  0.   ]
Mean Squared Error (MSE) trên tập test: 6.4693511759266515
Mean Squared Error (MSE) trên tập train: 6.651114995976714


In [306]:
train_data_lr['Hours_Studied^2'] = train_data_lr['Hours_Studied'] ** 2
train_data_lr['Attendance^2'] = train_data_lr['Attendance'] ** 2
train_data_lr['Sleep_Hours^2'] = train_data_lr['Sleep_Hours'] ** 2
train_data_lr['Previous_Scores^2'] = train_data_lr['Previous_Scores'] ** 2
train_data_lr['Tutoring_Sessions^2'] = train_data_lr['Tutoring_Sessions'] ** 2
train_data_lr['Physical_Activity^2'] = train_data_lr['Physical_Activity'] ** 2

test_data_lr['Hours_Studied^2'] = test_data_lr['Hours_Studied'] ** 2
test_data_lr['Attendance^2'] = train_data_lr['Attendance'] ** 2
test_data_lr['Sleep_Hours^2'] = train_data_lr['Sleep_Hours'] ** 2
test_data_lr['Previous_Scores^2'] = train_data_lr['Previous_Scores'] ** 2
test_data_lr['Tutoring_Sessions^2'] = train_data_lr['Tutoring_Sessions'] ** 2
test_data_lr['Physical_Activity^2'] = train_data_lr['Physical_Activity'] ** 2


In [307]:
y_train = train_data_lr["Exam_Score"].values
y_test = test_data_lr["Exam_Score"].values 
    
mse_results = []

for column in train_data_lr.columns[:-1]:
    if column == "Exam_Score":
        continue
    X_train_current = train_data_lr[[column]].values
    X_test_current = test_data_lr[[column]].values

    X_train_current = add_intercept(X_train_current)
    X_test_current = add_intercept(X_test_current)
    
    w = train_linreg(X_train_current, y_train)

    y_pred = predict_linreg(X_test_current, w)

    mse = loss_function(y_pred, y_test)
    mse_results.append((column, mse))

mse_results.sort(key=lambda x: x[1])
features = mse_results[:6]

top_features = []

for i, (feature, mse) in enumerate(features):
    print(f"{feature}, MSE = {mse:.4f}")
    top_features.append(feature)

Attendance, MSE = 9.9857
Hours_Studied, MSE = 12.3517
Hours_Studied^2, MSE = 12.4548
Previous_Scores, MSE = 14.9169
Tutoring_Sessions, MSE = 15.0533
Sleep_Hours^2, MSE = 15.5323


Ta xây dựng hàm tìm lỗi MSE cho từng thuộc tính, từ đó chọn ra 5 thuộc tính có lỗi MSE thấp nhất:
- Attendance, MSE = 9.9857
- Hours_Studied, MSE = 12.3517
- Hours_Studied^2, MSE = 12.4548
- Previous_Scores, MSE = 14.9169
- Tutoring_Sessions, MSE = 15.0533
- Sleep_Hours^2, MSE = 15.5323
Chọn lần lượt 2, 3, 4, 5, 6 thuộc tính đầu tiên trong 5 thuộc tính này để xây dựng 5 cây Decision tree tương ứng

In [308]:
def linear_regress_n_features(top_features, n, train_data_lr, test_data_lr, y_train, y_test):
    selected_features = top_features[:n]

    X_train_selected = train_data_lr[selected_features].values
    X_test_selected = test_data_lr[selected_features].values
    
    X_train_selected = add_intercept(X_train_selected)
    X_test_selected = add_intercept(X_test_selected)
    
    w = train_linreg(X_train_selected, y_train)
    
    y_pred = predict_linreg(X_test_selected, w)
    mse = loss_function(y_test, y_pred)
    
    print(f"Mô hình với {n} thuộc tính: {', '.join(selected_features)}")
    print("> Weights (w):", w.round(3))
    print("> Mean Squared Error (MSE):", mse)

In [309]:
for i in range(1, 7):
    linear_regress_n_features(top_features, i, train_data_lr, test_data_lr, y_train, y_test)

Mô hình với 1 thuộc tính: Attendance
> Weights (w): [51.678  0.195]
> Mean Squared Error (MSE): 9.98570816509502
Mô hình với 2 thuộc tính: Attendance, Hours_Studied
> Weights (w): [45.657  0.197  0.293]
> Mean Squared Error (MSE): 7.0176793478985084
Mô hình với 3 thuộc tính: Attendance, Hours_Studied, Hours_Studied^2
> Weights (w): [4.6186e+01 1.9700e-01 2.3500e-01 1.0000e-03]
> Mean Squared Error (MSE): 7.013354211988052
Mô hình với 4 thuộc tính: Attendance, Hours_Studied, Hours_Studied^2, Previous_Scores
> Weights (w): [4.2531e+01 1.9800e-01 2.4400e-01 1.0000e-03 4.6000e-02]
> Mean Squared Error (MSE): 6.4764743013016535
Mô hình với 5 thuộc tính: Attendance, Hours_Studied, Hours_Studied^2, Previous_Scores, Tutoring_Sessions
> Weights (w): [4.1714e+01 1.9800e-01 2.5100e-01 1.0000e-03 4.7000e-02 4.9100e-01]
> Mean Squared Error (MSE): 6.062430327306808
Mô hình với 6 thuộc tính: Attendance, Hours_Studied, Hours_Studied^2, Previous_Scores, Tutoring_Sessions, Sleep_Hours^2
> Weights (w): 